# Chapter 3

## 3.1

In [6]:
(define (make-accumulator acc)
  (lambda (x) (set! acc (+ acc x)) acc))

In [7]:
(define A (make-accumulator 5))

In [8]:
(A 10)

15

In [10]:
(A 10)

25

## 3.2

In [15]:
(define (make-monitored f)
  (define count 0)
  (define (mf m)
    (cond ((eq? m 'how-many-calls?) count)
          ((eq? m 'reset-count) (set! count 0))
          (else 
           (set! count (+ count 1))
           (f m))))
  mf)

In [30]:
(define s (make-monitored sqrt))

In [31]:
(s 100)

10.0

In [32]:
(s 'how-many-calls?)

1

In [33]:
(s 25)

5.0

In [34]:
(s 'how-many-calls?)

2

In [35]:
(s 'reset-count)
(s 'how-many-calls?)

0

## 3.3

In [79]:
(define (make-account balance password)
  (define (withdraw amount)
    (if (>= balance amount)
        (begin (set! balance
                     (- balance amount))
               balance)
        "Insufficient funds"))
  (define (deposit amount)
    (set! balance (+ balance amount))
    balance)
  
  (define (dispatch m)
    (cond ((eq? m 'withdraw) withdraw)
          ((eq? m 'deposit) deposit)
          (else (error "Unkown request:
                       MAKE-ACCOUNT" m))))
  
  (define (authorise pass m)
    (if (eq? pass password)
        (dispatch m)
        (lambda (amount) "Incorrect password")))
    
  authorise)

In [80]:
(define acc
  (make-account 150 '1234safepassword))

((acc '1234safepassword 'withdraw) 100)

50

In [42]:
((acc 'wrongpassword 'deposit) 100)

"Incorrect password"

## 3.4

In [71]:
(define (make-account balance password)
  (define failed-attempts 0)
  (define (call-the-police) "Too many failed attempts, police on the way.")
  (define (withdraw amount)
    (if (>= balance amount)
        (begin (set! balance
                     (- balance amount))
               balance)
        "Insufficient funds"))
  (define (deposit amount)
    (set! balance (+ balance amount))
    balance)
  
  (define (dispatch m)
    (cond ((eq? m 'withdraw) withdraw)
          ((eq? m 'deposit) deposit)
          (else (error "Unkown request:
                       MAKE-ACCOUNT" m))))
  
  (define (authorise pass m)
    (if (eq? pass password)
        (begin (set! failed-attempts 0)
               (dispatch m))
        (begin (set! failed-attempts (+ 1 failed-attempts))
               (lambda (amount) "Incorrect password"))))
  
  (define (limit-attempts pass m)
    (if (>= failed-attempts 7)
         (lambda (x) (call-the-police))
        (authorise pass m)))
    
  limit-attempts)

In [72]:
(define acc
  (make-account 150 '1234safepassword))

In [73]:
((acc '1234safepassword 'withdraw) 100)

50

In [74]:
((acc 'wrongpassword 'withdraw) 100)
((acc 'wrongpassword 'withdraw) 100)
((acc 'wrongpassword 'withdraw) 100)
((acc 'wrongpassword 'withdraw) 100)
((acc 'wrongpassword 'withdraw) 100)
((acc 'wrongpassword 'withdraw) 100)
((acc 'wrongpassword 'withdraw) 100)

((acc '1234safepassword 'withdraw) 100)

"Too many failed attempts, police on the way."

## 3.5

In [15]:
(define (square x)
  (* x x))

(define (monte-carlo trials experiment)
  (define (iter trials-remaining trials-passed)
    (cond ((= trials-remaining 0)
           ( / trials-passed trials))
          ((experiment)
           (iter (- trials-remaining 1)
                 (+ trials-passed 1)))
          (else
           (iter (- trials-remaining 1)
                 trials-passed))))
  (iter trials 0))

(define (random-in-range low high)
  (let ((range (- high low)))
    (+ low (random range))))

(define (make-point x y) (cons x y))
(define (get-x point) (car point))
(define (get-y point) (cdr point))

(define (make-circle center-point radius)
   (list center-point radius))
(define (get-center circle) (car circle))
(define (get-radius circle) (cadr circle))

(define (make-rect p1 p2) (list p1 p2))
(define (width-rect rect)
  (abs (- (get-x (car rect))
          (get-x (cadr rect)))))
(define (height-rect rect)
  (abs (- (get-y (car rect))
          (get-y (cadr rect)))))
(define (area-rect rect)
  (* (width-rect rect) (height-rect rect)))

(define (rand-point-in-rect rect)
  (make-point (random-in-range (get-x (car rect)) 
                               (get-x (cadr rect)))
              (random-in-range (get-y (car rect))
                               (get-y (cadr rect)))))

(define (point-in-circle? point circle)
  (let ((circle-center-x (get-x (get-center circle)))
        (circle-center-y (get-y (get-center circle))))
    (<= (+ (square (- (get-x point) circle-center-x))
           (square (- (get-y point) circle-center-y)))
        (square (get-radius circle)))))

;; P should take a rectangle as input and test whether a random
;; point within that rectangle is within a desired circle
;; the circle is internal to P
(define (estimate-integral P bounds-rect trials)
  (monte-carlo trials (P bounds-rect)))

(define unit-rect (make-rect (make-point -1.0 -1.0)
                             (make-point 1.0 1.0)))
(define unit-circle (make-circle (make-point 0 0) 1.0))


(define (estimate-pi-integral trials)
  (define (rand-point-in-unit-circle bounds-rect) 
    (lambda ()
      (point-in-circle? (rand-point-in-rect bounds-rect)
                        unit-circle)))
  (* (area-rect unit-rect) 
     (estimate-integral rand-point-in-unit-circle unit-rect trials))) 


This takes a long time to run -> markdown cell to prevent accidentally running again.
```scheme
;;(estimate-pi-integral 100000)
;; 3.144
```

## 3.6

In [62]:
;; naive implementations for testing
;; +1 to prevent returning 0 -> random takes positive value
(define (random-init) (+ 1 (random 1000))) 
(define (rand-update x) (+ 1 (random x)))

(define rand
  (let ((x (random-init)))
    (lambda (m)
      (cond ((eq? m 'generate)
             (set! x (rand-update x)) x)
            ((eq? m 'reset)
             (lambda (new-value)
               (set! x new-value) x))
            (else (error "Unkown method:
                         RAND" m))))))

In [63]:
(rand 'generate)

338

In [64]:
((rand 'reset) 10)

10

## 3.7

In [74]:
(define (make-account balance password)
  (define (withdraw amount)
    (if (>= balance amount)
        (begin (set! balance
                     (- balance amount))
               balance)
        "Insufficient funds"))
  (define (deposit amount)
    (set! balance (+ balance amount))
    balance)
  
  (define (dispatch m)
    (cond ((eq? m 'withdraw) withdraw)
          ((eq? m 'deposit) deposit)
          (else (error "Unkown request:
                       MAKE-ACCOUNT" m))))
  
  (define (correct-password? pass)
    (eq? pass password))
  
  (define (authorise pass m)
    (if (correct-password? pass)
        (dispatch m)
        (lambda (amount) "Incorrect password")))
  authorise)

(define (make-joint acc password joint-password)
  (define (dispatch pass m)
    (cond ((not (eq? pass joint-password))
           (error "Incorrect password"))
          (else (acc password m))))
  dispatch)

In [75]:
(define peter-acc (make-account 200 'peters-password))
(define paul-acc
  (make-joint peter-acc
              'peters-password
              'pauls-password))

((peter-acc 'peters-password 'withdraw) 10)

190

In [76]:
((paul-acc 'pauls-password 'withdraw) 10)

180

In [77]:
((peter-acc 'peters-password 'withdraw) 10)

170

In [78]:
((paul-acc 'pauls-password 'deposit) 100)

270

In [81]:
((peter-acc 'peters-password 'withdraw) 10)

260

## 3.8
`f` should return it's argument if it has been previously called, else it should return `0`.

In [99]:
(define f
  (let ((called? #f))
    (lambda (x)
      (if called?
          0
          (begin
           (set! called? #t)
           x)))))

(+ (f 0) (f 1))

0

In [100]:
;; re-define to clear out internal state
(define f
  (let ((called? #f))
    (lambda (x)
      (if called?
          0
          (begin
           (set! called? #t)
           x)))))
;; call opposite way round to simulate changing evaluation order
(+ (f 1) (f 0))

1

## 3.9

### Recursive Factorial
```scheme
(define (factorial n)
  (if (= n 1)
      1
      (* n (factorial (- n 1)))))
```

#### Environment Structure
``` 
         __________________
global  |                  |
env --->|factorial: *      |
        |           |      |
        |___________|______|
                    |
            parameters: n
            body: (if (= n 1) 1 (* n (factorial (-n 1))))

(factorial 6)
             _______      ^
            |       |     | global env
    E1 ---> | n : 6 |_____|
            |_______|

(* 6 (factorial 5))
             _______      ^
            |       |     | global env
    E2 ---> | n : 5 |_____|
            |_______|
            
(* 5 (factorial 4))
             _______      ^
            |       |     | global env
    E3 ---> | n : 4 |_____|
            |_______|

(* 4 (factorial 3))
             _______      ^
            |       |     | global env
    E4 ---> | n : 3 |_____|
            |_______|

(* 3 (factorial 2))
             _______      ^
            |       |     | global env
    E5 ---> | n : 2 |_____|
            |_______|

(* 2 (factorial 1))
             _______      ^
            |       |     | global env
    E2 ---> | n : 1 |_____|
            |_______|
1
```

### Iterative Factorial
```scheme
(define (factorial n)
  (fact-iter 1 1 n))

(define (fact-iter product 
                   counter 
                   max-count)
  (if (> counter max-count)
      product
      (fact-iter (* counter product)
                 (+ counter 1)
                 max-count)))
```

#### Environment Structure
```
         ________________________________
global  |                                |
env --->|factorial: *                    |
        |fact-iter: |                 *  |
        |___________|_________________|__|
                    |                 |
            parameters: n             |
            body: (fact-iter 1 1 n)   parameters: (product counter max-count)
                                      body: (if (> counter max-count)
                                                  product
                                                  (fact-iter (* counter product)
                                                             (+ counter 1)
                                                             max-count))
(factorial 6)
             _______      ^
            |       |     | global env
    E1 ---> | n : 6 |_____|
            |_______|

(fact-iter 1 1 6)
             ______________      ^
            | product: 1   |     | global env
    E2 ---> | counter : 1  |_____|
            | max-count: 6 |
            |______________|

(fact-iter 1 2 6)
             ______________      ^
            | product: 1   |     | global env
    E3 ---> | counter : 2  |_____|
            | max-count: 6 |
            |______________|
            
(fact-iter 2 3 6)
             ______________      ^
            | product: 2   |     | global env
    E3 ---> | counter : 3  |_____|
            | max-count: 6 |
            |______________|

(fact-iter 6 4 6)
             ______________      ^
            | product: 6   |     | global env
    E3 ---> | counter : 4  |_____|
            | max-count: 6 |
            |______________|
            
(fact-iter 24 5 6)
             ______________      ^
            | product: 24  |     | global env
    E3 ---> | counter : 5  |_____|
            | max-count: 6 |
            |______________|

(fact-iter 120 6 6)
             ______________      ^
            | product: 120 |     | global env
    E3 ---> | counter : 6  |_____|
            | max-count: 6 |
            |______________|

(fact-iter 720 7 6)
             ______________      ^
            | product: 720 |     | global env
    E3 ---> | counter : 7  |_____|
            | max-count: 6 |
            |______________|
720
```

## 3.10

`(define W1 (make-withdraw 100))`:

Evaluating `make-withdraw` creates a new frame - `Frame A` within which `initial-amount` is bound to the value `100`.

Evaluating the lambda function within `make-withdraw` creates a new frame `Frame B` within which `balance` is bound to the value of `initial-amount` from `Frame A`.

```
         _______________________
global->| make-withdraw : *     |
env     | W1 :  *         |     |
         -------|---^-----|---^-
                |   |     |   |
                |   |     parameters: initial-amount
                |   |     body: ((lambda (balance) ((...))) initial-amount)
                |   |
                |  _|___Frame_A__________
                | | initial-amount : 100 | E1
                |  -^--------------------
                |   |
                |  _|___Frame_B___
                | | balance : 100 | E2
                |  -^-------------
                |   |
                parameters: balance
                body: (lambda (amount) (if (>= balance amount) ... ))
```


`(W1 50)`:

The call to `set!` within `Frame B` will affect the value of `balance` within `Frame B` and will not affect the value of `initial-amount` within `Frame A`.

```
         _______________________
global->| make-withdraw : *     |
env     | W1 :  *         |     |
         -------|---^-----|---^-
                |   |     |   |
                |   |     parameter: initial-amount
                |   |     body: ((lambda (balance) ((...))) initial-amount)
                |   |
                |  _|___Frame_A__________
                | | initial-amount : 100 | E1
                |  -^--------------------
                |   |
                |  _|__Frame_B___
                | | balance : 50 | E2
                |  -^------------
                |   |
                |  _|__Frame_C___
                | | amount : 50  | E3
                |  -^------------
                |   |
                parameters: amount
                body: (if (>= balance amount) ... )
```

## 3.12

Missing `<response>`s in order:
1. `(b)`
    - `(define x (list 'a 'b))` binds `x` to the value `(a b)`
    - `(cdr x)` simply returns the `cdr` of the list `(a b)`
2. `(b c d)`
    - `(define y (list 'c 'd))` binds `y` the the value `(c d)`
    - `(define w (append! x y))` binds the result of `(append! x y)` to `w`
    - `(append! x y)` *mutates* `x` such that the list `y` is appended onto it and *returns* a pointer to `x`.
    - `w` now points to the mutated value of `x`
    - `(set-cdr! (last-pair x) y)` within `append!` will set the `cdr` of the last pair in `x` to the list `y`
    - `(last-pair x)` evalutes to `(last-pair (a b)) ` which returns `(b)`
    - `(set-cdr! (last-pair x) y)` evaluates to `(set-cdr! (b) (c d))`, *mutating* `(b)` to produce `(b c d)`

### Demonstration

In [123]:
(define (append! x y)
  (set-cdr! (last-pair x) y)
  x)

(define (last-pair x)
  (if (null? (cdr x))
      x
  (last-pair (cdr x))))

(define x (list 'a 'b))
(define y (list 'c 'd))
(define z (append x y))

z

(a b c d)

In [124]:
(cdr x)

(b)

In [125]:
(define w (append! x y))
w

(a b c d)

In [126]:
(cdr x)

(b c d)